In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

In [ ]:
from PIL import Image
from PIL.Image import Dither

In [ ]:
#

In [ ]:
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
_ = pipe.to("cuda")

In [ ]:
#

In [ ]:
prompt = "mysterious forest, creatures, Thomas Nast"

In [ ]:
image = pipe(
        prompt,
        num_inference_steps=1,
        guidance_scale=0.0,
        num_images_per_prompt=1,
        width=960,
        height=680
    ).images[0]

In [ ]:
image

## Try dithering and send to 

In [ ]:
image.convert("1")

In [ ]:
image_f = image.convert("1", dither=Dither.FLOYDSTEINBERG)
image_f

In [ ]:
def atkinson(image: Image.Image):
    img = image.convert('L')
    threshold = 128*[0] + 128*[255]
    for y in range(img.size[1]):
        for x in range(img.size[0]):
    
            old = img.getpixel((x, y))
            new = threshold[old]
            err = (old - new) >> 3 # divide by 8
                
            img.putpixel((x, y), new)
            
            for nxy in [(x+1, y), (x+2, y), (x-1, y+1), (x, y+1), (x+1, y+1), (x, y+2)]:
                try:
                    img.putpixel(nxy, img.getpixel(nxy) + err)
                except IndexError:
                    pass
    return img

In [ ]:
image_a = atkinson(image)

In [ ]:
image_a

In [ ]:
##

In [ ]:
# Get dimensions
width, height = image.size

# Find center
center_x = int(width/2)
center_y = int(height/2)

# Split two images
a = image_f.crop((0, 0, center_x, height))
b = image_a.crop((center_x, 0, width, height))

# Merge into one
image_m = image.copy()
image_m.paste(a, (0,0))
image_m.paste(b, (center_x,0))

In [ ]:
image_m

In [ ]:
##

In [ ]:
from art_utils import network_utils

In [ ]:
r = network_utils.send_photo(image_m, "http://192.168.1.26:8080/display/bitmap")
r